# Deep Learning avec PyTorch

## Fonctions Dérivables

Supports adaptés de Sylvain Lamprier (sylvain.lamprier@univ-angers.fr), Nicolas Baskiotis (nicolas.baskiotis@sorbonne-univeriste.fr) et Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

In [1]:
import torch
print("La version de torch est : ",torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())

import numpy as np
import sklearn

La version de torch est :  2.1.0
Le calcul GPU est disponible ?  False


Dans la partie précédente, nous avons vu comment implémenter une regression linéaire en utilisant les structures Tensor de PyTorch. Cependant, nous exploitions pas du tout la puissance de PyTorch qui permet de faciliter le calcul des gradients via de l'auto-dérivation. Dans la partie précédente précédent nous avions défini un algorithme spécifique à de la regression pour un modèle (linéaire) et un coût (moindres carrés) figés, en définissant à la main le gradient du coût global pour l'ensemble des paramètres. Ce mode de programmation est très peu modulaire et est très difficilement étendable à des architectures plus complexes. Sachant que l'objectif est de développer des architectures neuronales avec des nombreux modules neuronaux enchaînés, il n'est pas possible de travailler de cette façon. 

Nous allons voir comment décomposer les choses pour rendre le code plus facilement généralisable. L'objectif est de comprendre le fonctionnement interne de PyTorch (sans en utiliser encore les facilités offertes par l'utilisation d'un graphe de calcul), basé sur l'implémentation d'objets Function.  

## Fonctions

PyTorch utilise une classe abstraite Function dont sont héritées toutes les fonctions et qui nécessite l'implémentation de ces deux méthodes :

- méthode $\texttt{forward(ctx, *inputs)}$ : calcule le résultat de l'application de la fonction
- méthode $\texttt{backward(ctx, *grad-outputs)}$ : calcule le gradient partiel par rapport à chaque entrée de la méthode $\texttt{forward}$; le nombre de $\texttt{grad-outputs}$ doit être égale aux nombre de sorties de $\texttt{forward}$ et le nombre de sorties doit être égale aux nombres de $\texttt{inputs}$ de $\texttt{forward}$.

Pour des raisons d'implémentation, les deux méthodes doivent être statiques. Le premier paramètre $\texttt{ctx}$ permet de sauvegarder un contexte lors de la passe $\texttt{forward}$ (par exemple les tenseurs d'entrées) et il est passé lors de la passe $\texttt{backward}$ en paramètre afin de récupérer les valeurs. $\textbf{Attention : }$ le contexte doit être unique pour chaque appel de $\texttt{forward}$.

Création des modules MSE (coût moindres carrés) et Linéaire. 

In [2]:
import torch
from torch.autograd import Function
from torch.autograd import gradcheck

class Context:
    """Un objet contexte très simplifié pour simuler PyTorch
    
    Un contexte différent doit être utilisé à chaque forward
    """
    def __init__(self):
        self._saved_tensors = ()

    def save_for_backward(self, *args):
        self._saved_tensors = args

    @property
    def saved_tensors(self):
        return self._saved_tensors


class MSE(Function):
    """Début d'implémentation de la fonction MSE"""
    @staticmethod
    def forward(ctx, yhat, y):
        ## Garde les valeurs nécessaires pour le backward
        ctx.save_for_backward(yhat, y)
        return torch.pow(yhat - y, 2).mean()

    @staticmethod
    def backward(ctx, grad_output):
        ## Calcul du gradient du module par rapport à chaque groupe d'entrées
        yhat, y = ctx.saved_tensors
        grad_yhat = 2 * (yhat - y) / yhat.numel()
        grad_y = -2 * (yhat - y) / yhat.numel()
        return grad_yhat * grad_output, grad_y * grad_output


class Linear(Function):
    """Début d'implémentation de la fonction Linear"""
    @staticmethod
    def forward(ctx, X, W, b):
        ctx.save_for_backward(X, W, b)
        return (X @ W) + b

    @staticmethod
    def backward(ctx, grad_output):
        X, W, b = ctx.saved_tensors
        grad_X = grad_output @ W.T
        grad_W = (grad_output.t() @ X).t()
        grad_b = grad_output.sum(0)
        return grad_X, grad_W, grad_b


# Utiliser gradcheck

mse = MSE.apply
linear = Linear.apply


In [3]:
# Test du gradient de MSE 
yhat = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
y = torch.randn(10,5, requires_grad=True, dtype=torch.float64)
torch.autograd.gradcheck(mse, (yhat, y))

True

In [4]:
# Test du gradient de Linear (sur le même modèle que MSE)

x = torch.randn(13, 5,requires_grad=True,dtype=torch.float64)
w = torch.randn(5, 7,requires_grad=True,dtype=torch.float64)
b = torch.randn(7,requires_grad=True,dtype=torch.float64)
torch.autograd.gradcheck(linear,(x,w,b))

True

## Descente de Gradient

Régression linéaire en utilisant les objets Function déclarés ci-dessus. 

In [5]:
## Chargement des données Boston et transformation en tensor.
from sklearn.datasets import fetch_california_housing
boston = fetch_california_housing() ## chargement des données
x = torch.tensor(boston['data'],dtype=torch.float)
y = torch.tensor(boston['target'],dtype=torch.float).view(-1,1)

print("Nombre d'exemples : ",x.size(0), "Dimension : ",x.size(1))
print("Nom des attributs : ", ", ".join(boston['feature_names']))

#initialisation aléatoire de w et b
w = torch.randn(x.size(1),1)
b =  torch.randn(1,1)

EPOCHS = 50000
EPS = 1e-7
for n_iter in range(EPOCHS):
    ## Calcul du forward (loss), avec création de nouveaux Context pour chaque module
    ctx_mse = Context()
    ctx_linear = Context()

    y_pred = Linear.forward(ctx_linear, x, w, b)
    loss = MSE.forward(ctx_mse, y_pred, y)

    if n_iter % 100 == 0:
        print(f"Itérations {n_iter}: loss {loss.item()}")

    ## Calcul du backward (grad_w, grad_b)
    grad_mse = MSE.backward(ctx_mse,1)
    grad_X, grad_W, grad_b = Linear.backward(ctx_linear, grad_mse[0])

    ## Mise à jour des paramètres du modèle
    #x -= EPS * grad_X
    w -= EPS * grad_W  # Update weights
    b -= EPS * grad_b  # Update bias

Nombre d'exemples :  20640 Dimension :  8
Nom des attributs :  MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude
Itérations 0: loss 2788145.5
Itérations 100: loss 4256.62841796875
Itérations 200: loss 3293.343505859375
Itérations 300: loss 2550.746337890625
Itérations 400: loss 1978.269775390625
Itérations 500: loss 1536.9329833984375
Itérations 600: loss 1196.686279296875
Itérations 700: loss 934.3665771484375
Itérations 800: loss 732.1173706054688
Itérations 900: loss 576.1751708984375
Itérations 1000: loss 455.9288635253906
Itérations 1100: loss 363.199462890625
Itérations 1200: loss 291.68206787109375
Itérations 1300: loss 236.5161590576172
Itérations 1400: loss 193.9551239013672
Itérations 1500: loss 161.1110382080078
Itérations 1600: loss 135.75746154785156
Itérations 1700: loss 116.1781997680664
Itérations 1800: loss 101.0504379272461
Itérations 1900: loss 89.35417938232422
Itérations 2000: loss 80.30341339111328
Itérations 2100: loss 73.2920608520